# Change in the number of pharmacies in health professional shortage areas

This notebook measures the number of open New York State pharmacies in health professional shortage areas (HPSAs) a federal designation by the Health Resources & Services Administration given to areas lacking in primary care services. The HPSA shapefile was downloaded from the HRSA and then filtered to only include designations in New York State before being read into this notebook.    

Open pharmacies from June 2012, 2017 and 2022 were analyzed to look at percent change over five- and ten-year periods across the state as well as within and outside New York City. Pharmacy data is also broken down by category to allow for analysis of change in the number of chain and independent pharmacies in the state.

In [14]:
import pandas as pd
import geopandas as gpd
import cenpy as cen
from getpass import getpass

In [15]:
pd.set_option("display.max_columns", None)

In [16]:
CENSUS_API_KEY = getpass('Enter your Census API Key: ')

Enter your Census API Key: ········


## Loading in pharmacy and population data

In [17]:
#Read in categorized pharmacy data
pharmacies_categorized = pd.read_csv('data/pharmacies_post_04_categorized.csv', parse_dates=['date_first_registered',
                                                                                        'registration_begins',
                                                                                        'registration_ends'])

In [18]:
#Geodataframe for pharmacy data
pharmacies_cat_geoframe = gpd.GeoDataFrame(pharmacies_categorized, geometry=gpd.points_from_xy\
                                       (pharmacies_categorized.displayLongitude, pharmacies_categorized.displayLatitude),\
                                       crs='epsg:4269')

In [19]:
#Three frames one for each year
pharmacies_cat_geoframe_22 = pharmacies_cat_geoframe.query('(registration_ends > "2022-06-21") & (date_first_registered < "2022-06-21")')
pharmacies_cat_geoframe_17 = pharmacies_cat_geoframe.query('(registration_ends > "2017-06-21") & (date_first_registered < "2017-06-21")')
pharmacies_cat_geoframe_12 = pharmacies_cat_geoframe.query('(registration_ends > "2012-06-21") & (date_first_registered < "2012-06-21")')

In [20]:
# Getting population data to contextualize how widespread HPSAs are
con = cen.remote.APIConnection('ACSDT5Y2020',apikey=CENSUS_API_KEY)

columns = [
    'B01003_001' # Total Population
]

columns_E = [i+'E' for i in columns]
columns_M = [i+'M' for i in columns]

g_unit = 'tract'
g_filter = {'state':'36'} 

tract_pop_20 = con.query(columns_E + columns_M, geo_unit=g_unit, geo_filter=g_filter)

In [21]:
# Getting population data to contextualize how widespread HPSAs are
con = cen.remote.APIConnection('ACSDT5Y2020',apikey=CENSUS_API_KEY)

columns = [
    'B01003_001' # Total Population
]

columns_E = [i+'E' for i in columns]
columns_M = [i+'M' for i in columns]

g_unit = 'county subdivision'
g_filter = {'state':'36'} 

sub_pop_20 = con.query(columns_E + columns_M, geo_unit=g_unit, geo_filter=g_filter)

In [22]:
nyc_fips = ['005', '047', '081', '085', '061']

## HPSA analysis

In [23]:
hpsa = gpd.read_file('data/nys_hpsas/')

In [24]:
hpsa = hpsa.query('StAbbr == "NY"')

In [25]:
hpsa.CmpStAbbr.value_counts()

NY    2419
Name: CmpStAbbr, dtype: int64

In [26]:
hpsa.CmpTypDes.value_counts()

Census Tract          2263
County Subdivision     118
Single County           38
Name: CmpTypDes, dtype: int64

In [27]:
hpsa_subdivs = list(hpsa.query("CmpTypDes == 'County Subdivision'").GeoID)

In [28]:
tract_pop_20['GEOID'] = tract_pop_20.state + tract_pop_20.county + tract_pop_20.tract
sub_pop_20['GEOID'] = sub_pop_20.state + sub_pop_20.county + sub_pop_20['county subdivision']

In [29]:
subdiv_hpsa_pop = sub_pop_20.query('GEOID.isin(@hpsa_subdivs)').B01003_001E.astype(float).sum()

In [30]:
total_pop = tract_pop_20.B01003_001E.astype(float).sum()

In [31]:
(subdiv_hpsa_pop / total_pop) * 100 # HPSA Subdivs represent only 2% of the state's population

2.146432186075332

In [32]:
hpsa.HpsTypDes.value_counts()

HPSA Population    2215
Geographic HPSA     204
Name: HpsTypDes, dtype: int64

In [33]:
hpsa.HpsDgnLUDT.min() # The HPSA updated the longest ago was in 2016, reflecting the data is fairly recent

'2016-12-16'

In [54]:
hpsa_designated_geoids = list(hpsa.query("HpsStatDes == 'Designated'").GeoID)

In [35]:
hpsa_pop = tract_pop_20[tract_pop_20.GEOID\
                        .str.startswith(tuple(hpsa_designated_geoids))].B01003_001E.astype(int).sum()

In [36]:
# 16% of the population lives in an HPSA. 
# Subdivs are only a small part of that, so we're not omitting a lot of data by excluding them
hpsa_pop / total_pop * 100 

41.22405456480857

In [37]:
hpsa.HpsStatDes.value_counts() 

Designated                 1582
Proposed For Withdrawal     837
Name: HpsStatDes, dtype: int64

In [38]:
hpsa = hpsa.query("HpsStatDes == 'Designated'") # Excluding HPSAs proposed for withdrawal

In [39]:
tracts21 = gpd.read_file('data/tl_2021_36_tract/')

In [42]:
# Tract level merge with pharmacy data
# Using the most recent census tracts for analysis
pharmacy_tracts_22 = gpd.sjoin(pharmacies_cat_geoframe_22, tracts21, predicate='intersects') 
pharmacy_tracts_17 = gpd.sjoin(pharmacies_cat_geoframe_17, tracts21, predicate='intersects') 
pharmacy_tracts_12 = gpd.sjoin(pharmacies_cat_geoframe_12, tracts21, predicate='intersects') 

# Creating a binary column for whether a pharmacy is in a HPSA
# Using the current designations for analysis
pharmacy_tracts_22.loc[pharmacy_tracts_22.GEOID.str.startswith(tuple(hpsa_designated_geoids)), 'is_hpsa'] = True
pharmacy_tracts_17.loc[pharmacy_tracts_17.GEOID.str.startswith(tuple(hpsa_designated_geoids)), 'is_hpsa'] = True
pharmacy_tracts_12.loc[pharmacy_tracts_12.GEOID.str.startswith(tuple(hpsa_designated_geoids)), 'is_hpsa'] = True

In [43]:
# Filling in any non-labed tract as a non-HPSA
pharmacy_tracts_22.is_hpsa = pharmacy_tracts_22.is_hpsa.fillna(False)
pharmacy_tracts_17.is_hpsa = pharmacy_tracts_17.is_hpsa.fillna(False)
pharmacy_tracts_12.is_hpsa = pharmacy_tracts_12.is_hpsa.fillna(False)

In [44]:
# Generating counts per tract
hpsa_cat_2022 = pharmacy_tracts_22.groupby(['GEOID', 'category', 'is_hpsa']).registration_number.nunique().reset_index(name='num_pharmacies_2022')
hpsa_cat_2017 = pharmacy_tracts_17.groupby(['GEOID', 'category', 'is_hpsa']).registration_number.nunique().reset_index(name='num_pharmacies_2017')
hpsa_cat_2012 = pharmacy_tracts_12.groupby(['GEOID', 'category', 'is_hpsa']).registration_number.nunique().reset_index(name='num_pharmacies_2012')

# Merging counts from all three years
hpsa_breakdown_cat = hpsa_cat_2022.merge(hpsa_cat_2017, on=['GEOID', 'category', 'is_hpsa'], how='outer')\
                                   .merge(hpsa_cat_2012, on=['GEOID', 'category', 'is_hpsa'], how='outer')

# Tracts with no pharmacies won't show up, fill them in as zeros
hpsa_breakdown_cat = hpsa_breakdown_cat.fillna(0)

hpsa_breakdown_cat['change_indep_10yr'] = hpsa_breakdown_cat.query("category == 'INDEPENDENT'").num_pharmacies_2022 - hpsa_breakdown_cat.query("category == 'INDEPENDENT'").num_pharmacies_2012
hpsa_breakdown_cat['change_indep_5yr'] = hpsa_breakdown_cat.query("category == 'INDEPENDENT'").num_pharmacies_2022 - hpsa_breakdown_cat.query("category == 'INDEPENDENT'").num_pharmacies_2017

hpsa_breakdown_cat['change_majchain_10yr'] = hpsa_breakdown_cat.query("category == 'MAJOR CHAIN'").num_pharmacies_2022 - hpsa_breakdown_cat.query("category == 'MAJOR CHAIN'").num_pharmacies_2012
hpsa_breakdown_cat['change_majchain_5yr'] = hpsa_breakdown_cat.query("category == 'MAJOR CHAIN'").num_pharmacies_2022 - hpsa_breakdown_cat.query("category == 'MAJOR CHAIN'").num_pharmacies_2017

hpsa_breakdown_cat.head()

,GEOID,category,is_hpsa,num_pharmacies_2022,num_pharmacies_2017,num_pharmacies_2012,change_indep_10yr,change_indep_5yr,change_majchain_10yr,change_majchain_5yr
0,36001000202,HEALTHCARE,False,1.0,0.0,0.0,NaN,NaN,NaN,NaN
1,36001000202,INDEPENDENT,False,1.0,1.0,0.0,1.0,0.0,NaN,NaN
2,36001000302,MAJOR CHAIN,False,1.0,3.0,3.0,NaN,NaN,-2.0,-2.0
3,36001000401,MAJOR CHAIN,False,1.0,1.0,1.0,NaN,NaN,0.0,0.0
4,36001000403,MAJOR CHAIN,False,2.0,2.0,2.0,NaN,NaN,0.0,0.0


In [114]:
# Writing hpsa data to a CSV
hpsa_breakdown_cat.to_csv('data/hpsa_breakdown_categorized_COUNTS.csv', index=False)
pharmacy_tracts_22.to_csv('data/hpsa_pharmacies_labeled_2022.csv', index=False)
pharmacy_tracts_17.to_csv('data/hpsa_pharmacies_labeled_2017.csv', index=False)

In [115]:
hpsa_cat_2022.is_hpsa.value_counts()

False    1987
True     1388
Name: is_hpsa, dtype: int64

## Percent change calculations

### Overall Change

In [116]:
# CHANGE OVERALL
# Five year change among all hpsas
(hpsa_breakdown_cat.query('is_hpsa').num_pharmacies_2022.sum() - hpsa_breakdown_cat.query('is_hpsa').num_pharmacies_2017.sum())\
/ hpsa_breakdown_cat.query('is_hpsa').num_pharmacies_2017.sum() * 100

0.31905195989061075

In [117]:
# Five year change among non-hpsas
(hpsa_breakdown_cat.query('is_hpsa == False').num_pharmacies_2022.sum() - hpsa_breakdown_cat.query('is_hpsa == False').num_pharmacies_2017.sum())\
/ hpsa_breakdown_cat.query('is_hpsa == False').num_pharmacies_2017.sum() * 100

-0.10291595197255575

In [118]:
# Five year change overall
(hpsa_breakdown_cat.num_pharmacies_2022.sum() - hpsa_breakdown_cat.num_pharmacies_2017.sum())\
/ hpsa_breakdown_cat.num_pharmacies_2017.sum() * 100

0.07829320806420043

#### NYC 

In [121]:
# CHANGE OVERALL - JUST NYC
hpsa_breakdown_nyc = hpsa_breakdown_cat[hpsa_breakdown_cat.GEOID.str[2:5].isin(nyc_fips)]

In [122]:
# Five year change among all hpsas
(hpsa_breakdown_nyc.query('is_hpsa').num_pharmacies_2022.sum() - hpsa_breakdown_nyc.query('is_hpsa').num_pharmacies_2017.sum())\
/ hpsa_breakdown_nyc.query('is_hpsa').num_pharmacies_2017.sum() * 100

3.818953323903819

In [123]:
# Five year change among all non-hpsas
(hpsa_breakdown_nyc.query('is_hpsa==False').num_pharmacies_2022.sum() - hpsa_breakdown_nyc.query('is_hpsa==False').num_pharmacies_2017.sum())\
/ hpsa_breakdown_nyc.query('is_hpsa==False').num_pharmacies_2017.sum() * 100

4.182509505703422

In [124]:
# Ten year change among all hpsas
(hpsa_breakdown_nyc.query('is_hpsa').num_pharmacies_2022.sum() - hpsa_breakdown_nyc.query('is_hpsa').num_pharmacies_2012.sum())\
/ hpsa_breakdown_nyc.query('is_hpsa').num_pharmacies_2012.sum() * 100

23.56902356902357

In [125]:
# Ten year change among all non-hpsas
(hpsa_breakdown_nyc.query('is_hpsa==False').num_pharmacies_2022.sum() - hpsa_breakdown_nyc.query('is_hpsa==False').num_pharmacies_2012.sum())\
/ hpsa_breakdown_nyc.query('is_hpsa==False').num_pharmacies_2012.sum() * 100

13.881961762261014

#### Outside NYC

In [126]:
hpsa_breakdown_not_nyc = hpsa_breakdown_cat[hpsa_breakdown_cat.GEOID.str[2:5].isin(nyc_fips)==False]

In [127]:
# Five year change among all hpsas
(hpsa_breakdown_not_nyc.query('is_hpsa').num_pharmacies_2022.sum() - hpsa_breakdown_not_nyc.query('is_hpsa').num_pharmacies_2017.sum())\
/ hpsa_breakdown_not_nyc.query('is_hpsa').num_pharmacies_2017.sum() * 100

-6.0256410256410255

In [128]:
# Five year change among all non-hpsas
(hpsa_breakdown_not_nyc.query('is_hpsa==False').num_pharmacies_2022.sum() - hpsa_breakdown_not_nyc.query('is_hpsa==False').num_pharmacies_2017.sum())\
/ hpsa_breakdown_not_nyc.query('is_hpsa==False').num_pharmacies_2017.sum() * 100

-3.6249999999999996

In [130]:
# Ten year change among all hpsas
(hpsa_breakdown_not_nyc.query('is_hpsa').num_pharmacies_2022.sum() - hpsa_breakdown_not_nyc.query('is_hpsa').num_pharmacies_2012.sum())\
/ hpsa_breakdown_not_nyc.query('is_hpsa').num_pharmacies_2012.sum() * 100

-8.7173100871731

In [131]:
# Ten year change among all non-hpsas
(hpsa_breakdown_not_nyc.query('is_hpsa==False').num_pharmacies_2022.sum() - hpsa_breakdown_not_nyc.query('is_hpsa==False').num_pharmacies_2012.sum())\
/ hpsa_breakdown_not_nyc.query('is_hpsa==False').num_pharmacies_2012.sum() * 100

-3.6249999999999996

### Independent pharmacies

In [132]:
# Five year percent change among all hpsas independents
hpsa_breakdown_cat.query('(is_hpsa) & (category=="INDEPENDENT")').change_indep_5yr.sum()\
/ hpsa_breakdown_cat.query('(is_hpsa) & (category=="INDEPENDENT")').num_pharmacies_2017.sum() * 100

6.217616580310881

In [133]:
# Five year percent change among all non-hpsas independents
hpsa_breakdown_cat.query('(is_hpsa == False) & (category=="INDEPENDENT")').change_indep_5yr.sum() \
/ hpsa_breakdown_cat.query('(is_hpsa == False) & (category=="INDEPENDENT")').num_pharmacies_2017.sum() * 100

10.08094186902134

In [134]:
# Five year percent change in independents over all
hpsa_breakdown_cat.query('(category=="INDEPENDENT")').change_indep_5yr.sum() \
/ hpsa_breakdown_cat.query('(category=="INDEPENDENT")').num_pharmacies_2017.sum() * 100

8.1549815498155

In [135]:
# Ten year percent change among all hpsas independents
hpsa_breakdown_cat.query('(is_hpsa) & (category=="INDEPENDENT")').change_indep_5yr.sum()\
/ hpsa_breakdown_cat.query('(is_hpsa) & (category=="INDEPENDENT")').num_pharmacies_2012.sum() * 100

7.59493670886076

In [136]:
# Ten year percent change among all non-hpsas independents
hpsa_breakdown_cat.query('(is_hpsa == False) & (category=="INDEPENDENT")').change_indep_5yr.sum() \
/ hpsa_breakdown_cat.query('(is_hpsa == False) & (category=="INDEPENDENT")').num_pharmacies_2012.sum() * 100

11.669505962521294

In [137]:
# Ten year percent change in independents over all
hpsa_breakdown_cat.query('(category=="INDEPENDENT")').change_indep_5yr.sum() \
/ hpsa_breakdown_cat.query('(category=="INDEPENDENT")').num_pharmacies_2012.sum() * 100

9.692982456140351

#### NYC

In [138]:
# Five year percent change among all hpsas independents
hpsa_breakdown_nyc.query('(is_hpsa) & (category=="INDEPENDENT")').change_indep_5yr.sum()\
/ hpsa_breakdown_nyc.query('(is_hpsa) & (category=="INDEPENDENT")').num_pharmacies_2017.sum() * 100

8.391608391608392

In [139]:
# Five year percent change among all non-hpsas independents
hpsa_breakdown_nyc.query('(is_hpsa == False) & (category=="INDEPENDENT")').change_indep_5yr.sum() \
/ hpsa_breakdown_nyc.query('(is_hpsa == False) & (category=="INDEPENDENT")').num_pharmacies_2017.sum() * 100

15.75178997613365

In [140]:
# Five year percent change among all independents
hpsa_breakdown_nyc.query('category=="INDEPENDENT"').change_indep_5yr.sum() \
/ hpsa_breakdown_nyc.query('category=="INDEPENDENT"').num_pharmacies_2017.sum() * 100

11.503531786074673

In [141]:
# Ten year precent change among all hpsas independents
hpsa_breakdown_nyc.query('(is_hpsa) & (category=="INDEPENDENT")').change_indep_10yr.sum() \
/ hpsa_breakdown_nyc.query('(is_hpsa) & (category=="INDEPENDENT")').num_pharmacies_2012.sum() * 100

38.392857142857146

In [142]:
# Ten year precent change among all non-hpsas independents
hpsa_breakdown_nyc.query('(is_hpsa == False) & (category=="INDEPENDENT")').change_indep_10yr.sum() \
/ hpsa_breakdown_nyc.query('(is_hpsa == False) & (category=="INDEPENDENT")').num_pharmacies_2012.sum() * 100

37.5886524822695

In [213]:
# Ten year percent change among all independents
hpsa_breakdown_nyc.query('category=="INDEPENDENT"').change_indep_10yr.sum() \
/ hpsa_breakdown_nyc.query('category=="INDEPENDENT"').num_pharmacies_2012.sum() * 100

38.038725796377264

#### Outside NYC

In [143]:
# Five year percent change among all hpsas independents
hpsa_breakdown_not_nyc.query('(is_hpsa) & (category=="INDEPENDENT")').change_indep_5yr.sum()\
/ hpsa_breakdown_not_nyc.query('(is_hpsa) & (category=="INDEPENDENT")').num_pharmacies_2017.sum() * 100

-5.797101449275362

In [144]:
# Five year percent change among all non-hpsas independents
hpsa_breakdown_not_nyc.query('(is_hpsa == False) & (category=="INDEPENDENT")').change_indep_5yr.sum() \
/ hpsa_breakdown_not_nyc.query('(is_hpsa == False) & (category=="INDEPENDENT")').num_pharmacies_2017.sum() * 100

0.9596928982725527

In [145]:
# Five year percent change among all independents
hpsa_breakdown_not_nyc.query('(category=="INDEPENDENT")').change_indep_5yr.sum()\
/ hpsa_breakdown_not_nyc.query('(category=="INDEPENDENT")').num_pharmacies_2017.sum() * 100

-0.9615384615384616

In [146]:
# Ten year precent change among all hpsas independents
hpsa_breakdown_not_nyc.query('(is_hpsa) & (category=="INDEPENDENT")').change_indep_10yr.sum() \
/ hpsa_breakdown_not_nyc.query('(is_hpsa) & (category=="INDEPENDENT")').num_pharmacies_2012.sum() * 100

-7.142857142857142

In [147]:
# Ten year precent change among all non-hpsas independents
hpsa_breakdown_not_nyc.query('(is_hpsa == False) & (category=="INDEPENDENT")').change_indep_10yr.sum() \
/ hpsa_breakdown_not_nyc.query('(is_hpsa == False) & (category=="INDEPENDENT")').num_pharmacies_2012.sum() * 100

12.153518123667377

In [214]:
# Ten year percent change among all independents
hpsa_breakdown_not_nyc.query('(category=="INDEPENDENT")').change_indep_10yr.sum()\
/ hpsa_breakdown_not_nyc.query('(category=="INDEPENDENT")').num_pharmacies_2012.sum() * 100

6.185567010309279

### Major chains

In [148]:
# Five year percent change among all hpsas major chain
hpsa_breakdown_cat.query('(is_hpsa) & (category=="MAJOR CHAIN")').change_majchain_5yr.sum()\
/ hpsa_breakdown_cat.query('(is_hpsa) & (category=="MAJOR CHAIN")').num_pharmacies_2017.sum() * 100

-12.732474964234623

In [149]:
# Five year percent change among all non-hpsas major chain
hpsa_breakdown_cat.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').change_majchain_5yr.sum() \
/ hpsa_breakdown_cat.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').num_pharmacies_2017.sum() * 100

-11.37546468401487

In [150]:
# Five year percent change among all MAJOR CHAINs
hpsa_breakdown_cat.query('category=="MAJOR CHAIN"').change_majchain_5yr.sum() \
/ hpsa_breakdown_cat.query('category=="MAJOR CHAIN"').num_pharmacies_2017.sum() * 100

-11.839530332681017

In [76]:
# Ten year percent change among all hpsas major chain
hpsa_breakdown_cat.query('(is_hpsa) & (category=="MAJOR CHAIN")').change_majchain_10yr.sum()\
/ hpsa_breakdown_cat.query('(is_hpsa) & (category=="MAJOR CHAIN")').num_pharmacies_2012.sum() * 100

-17.67881241565452

In [77]:
# Ten year percent change among all non-hpsas major chain
hpsa_breakdown_cat.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').change_majchain_10yr.sum() \
/ hpsa_breakdown_cat.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').num_pharmacies_2012.sum() * 100

-15.75971731448763

In [78]:
# Ten year percent change among all MAJOR CHAINs
hpsa_breakdown_cat.query('category=="MAJOR CHAIN"').change_majchain_10yr.sum() \
/ hpsa_breakdown_cat.query('category=="MAJOR CHAIN"').num_pharmacies_2012.sum() * 100

-16.41929499072356

#### NYC

In [151]:
# Five year percent change among all hpsas major chain
hpsa_breakdown_nyc.query('(is_hpsa) & (category=="MAJOR CHAIN")').change_majchain_5yr.sum()\
/ hpsa_breakdown_nyc.query('(is_hpsa) & (category=="MAJOR CHAIN")').num_pharmacies_2017.sum() * 100

-20.642201834862387

In [152]:
# Five year percent change among all non-hpsas major chain
hpsa_breakdown_nyc.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').change_majchain_5yr.sum() \
/ hpsa_breakdown_nyc.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').num_pharmacies_2017.sum() * 100

-20.048899755501225

In [153]:
# Five year percent change among all MAJOR CHAINs
hpsa_breakdown_nyc.query('category=="MAJOR CHAIN"').change_majchain_5yr.sum() \
/ hpsa_breakdown_nyc.query('category=="MAJOR CHAIN"').num_pharmacies_2017.sum() * 100

-20.25518341307815

In [154]:
# Ten year precent change among all hpsas MAJOR CHAINs
hpsa_breakdown_nyc.query('(is_hpsa) & (category=="MAJOR CHAIN")').change_majchain_10yr.sum() \
/ hpsa_breakdown_nyc.query('(is_hpsa) & (category=="MAJOR CHAIN")').num_pharmacies_2012.sum() * 100

-27.916666666666668

In [155]:
# Ten year precent change among all non-hpsas MAJOR CHAINs
hpsa_breakdown_nyc.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').change_majchain_10yr.sum() \
/ hpsa_breakdown_nyc.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').num_pharmacies_2012.sum() * 100

-25.171624713958813

In [215]:
# Five year percent change among all MAJOR CHAINs
hpsa_breakdown_nyc.query('category=="MAJOR CHAIN"').change_majchain_10yr.sum() \
/ hpsa_breakdown_nyc.query('category=="MAJOR CHAIN"').num_pharmacies_2012.sum() * 100

-26.14475627769572

#### Outside NYC

In [156]:
# Five year percent change among all hpsas MAJOR CHAINs
hpsa_breakdown_not_nyc.query('(is_hpsa) & (category=="MAJOR CHAIN")').change_majchain_5yr.sum()\
/ hpsa_breakdown_not_nyc.query('(is_hpsa) & (category=="MAJOR CHAIN")').num_pharmacies_2017.sum() * 100

-9.147609147609149

In [157]:
# Five year percent change among all non-hpsas MAJOR CHAINs
hpsa_breakdown_not_nyc.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').change_majchain_5yr.sum() \
/ hpsa_breakdown_not_nyc.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').num_pharmacies_2017.sum() * 100

-7.585470085470085

In [158]:
# Five year percent change among all MAJOR CHAINs
hpsa_breakdown_not_nyc.query('(category=="MAJOR CHAIN")').change_majchain_5yr.sum()\
/ hpsa_breakdown_not_nyc.query('(category=="MAJOR CHAIN")').num_pharmacies_2017.sum() * 100

-8.115737473535638

In [159]:
# Ten year precent change among all hpsas MAJOR CHAINs
hpsa_breakdown_not_nyc.query('(is_hpsa) & (category=="MAJOR CHAIN")').change_majchain_10yr.sum() \
/ hpsa_breakdown_not_nyc.query('(is_hpsa) & (category=="MAJOR CHAIN")').num_pharmacies_2012.sum() * 100

-12.77445109780439

In [160]:
# Ten year precent change among all non-hpsas MAJOR CHAINs
hpsa_breakdown_not_nyc.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').change_majchain_10yr.sum() \
/ hpsa_breakdown_not_nyc.query('(is_hpsa == False) & (category=="MAJOR CHAIN")').num_pharmacies_2012.sum() * 100

-11.554192229038856

In [216]:
# Five year percent change among all MAJOR CHAINs
hpsa_breakdown_not_nyc.query('(category=="MAJOR CHAIN")').change_majchain_10yr.sum()\
/ hpsa_breakdown_not_nyc.query('(category=="MAJOR CHAIN")').num_pharmacies_2012.sum() * 100

-11.967545638945234